In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex 客户端库：AutoML 图像分割模型用于批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_segmentation_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_segmentation_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来创建图像分割模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行批量预测。

### 数据集

本教程使用的数据集是[TODO](https://)。这个数据集不需要任何特征工程。您在本教程中将使用的数据集版本存储在一个公共云存储桶中。

### 目标

在本教程中，您将从 Python 脚本创建一个 AutoML 图像分割模型，然后使用 Vertex 客户端库进行批量预测。您还可以选择使用 `gcloud` 命令行工具或在 Google Cloud 控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间存在一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并即时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，将结果存储在云存储存储桶中，准备就绪时返回。

成本

本教程使用Google Cloud（GCP）的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[定价 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google*云存储*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

*如果有这个选项，确保你在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得 300 美元的免费信用额度，用于支付计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对此笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 会将以 `!` 开头的行作为 shell 命令运行，并将以 `$` 开头的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改`REGION`变量，该变量用于整个笔记本的操作。以下是 Vertex 支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多地区存储桶来训练 Vertex。并非所有地区都支持所有 Vertex 服务。有关每个地区的最新支持，请参阅[Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您参加了一次直播教程会话，您可能正在使用一个共享的测试帐户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，则您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行身份验证，使用oAuth。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务帐号**。

在**服务帐号名称**字段中输入名称，然后点击**创建**。

在**授予此服务帐号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，选择**Vertex Administrator**。在过滤框中输入"Storage Object Admin"，选择**Storage Object Admin**。

点击创建。一个包含密钥的JSON文件将下载到您的本地环境中。

将您的服务帐号密钥的路径输入到下面单元格中作为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行此单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您的笔记本环境如何，以下步骤都是必需的。**

本教程旨在使用存储在公共云存储桶中的训练数据，并为您的批量预测使用本地云存储桶。您也可以使用您在本地云存储桶中存储的自己的训练数据。

在下面设置您的云存储桶的名称。存储桶名称必须在所有Google Cloud项目中具有全局唯一性，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容，验证对其的访问。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入顶点客户端库

将顶点客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、管道和端点服务的顶点API服务端点。
- `PARENT`：数据集、模型、作业、管道和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 自动机器学习常量

设置特定于AutoML数据集和训练的常量：

- 数据集模式：告诉`数据集`资源服务数据集的类型。
- 数据标签（注释）模式：告诉`数据集`资源服务数据如何标记（注释）。
- 数据集训练模式：告诉`管道`资源服务为何种任务（例如，分类）训练模型。

In [ ]:
# Image Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_segmentation_io_format_1.0.0.yaml"
# Image Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_segmentation_1.0.0.yaml"

硬件加速器

设置硬件加速器（例如，GPU）用于预测。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像，以及分配给虚拟机实例的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器映像，您应该指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
- aip.AcceleratorType.NVIDIA_TESLA_K80
- aip.AcceleratorType.NVIDIA_TESLA_P100
- aip.AcceleratorType.NVIDIA_TESLA_P4
- aip.AcceleratorType.NVIDIA_TESLA_T4
- aip.AcceleratorType.NVIDIA_TESLA_V100

否则，请指定`(None, None)`以使用一个运行在CPU上的容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

Docker容器镜像

对于AutoML批处理预测，预测服务将预先确定用于服务二进制文件的容器镜像。更具体地说，根据您选择的硬件加速器，该服务将为模型选择合适的容器。

#### 机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个 vCPU 3.75GB 的内存。
     - `n1-highmem`：每个 vCPU 6.5GB 的内存
     - `n1-highcpu`：每个 vCPU 0.9GB 的内存
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96\] 个CPU核心

*注意：您也可以在训练和部署时使用 n2 和 e2 机器类型，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在，您已经准备好开始创建自己的AutoML图像分割模型了。

## 设置客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本），您将创建一个客户端，该客户端向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此请提前设置它们。

- 用于`Dataset`资源的 Dataset 服务。
- 用于`Model`资源的 Model 服务。
- 用于训练的 Pipeline 服务。
- 用于批处理预测和自定义训练的 Job 服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户准备就绪，训练模型的第一步是创建一个管理的数据集实例，然后将标记数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建一个 `Dataset` 资源的实例。该函数执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个 Vertex `Dataset` 资源（`aip.Dataset`），具有以下参数：
 - `display_name`：您选择的可读名称。
 - `metadata_schema_uri`：数据集类型的架构。
3. 调用客户端数据集服务方法 `create_dataset`，具有以下参数：
 - `parent`：用于您的 `Database`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
 - `dataset`：您创建的 Vertex 数据集对象实例。
4. 该方法返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的方式。虽然这一步通常很快，但当您第一次在项目中使用它时，由于提供，可能会有较长的延迟。

您可以使用 `operation` 对象来获取操作的状态（例如，创建 `Dataset` 资源），或取消操作，通过调用操作方法：

| 方法         | 描述                  |
| ----------- | ----------- |
| result()    | 等待操作完成并以 JSON 格式返回结果对象。      |
| running()   | 返回操作是否仍在运行的 True/False。        |
| done()      | 返回操作是否已完成的 True/False。 |
| canceled()  | 返回操作是否已取消的 True/False。 |
| cancel()    | 取消操作（可能需要最多 30 秒）。 |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("unknown-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

Vertex的`Dataset`资源针对图像数据有一些要求：

- 图像必须存储在Cloud Storage桶中。
- 每个图像文件必须是图像格式（PNG，JPEG，BMP，...）。
- 必须在您的Cloud Storage桶中存储一个包含每个图像路径和标签的索引文件。
- 索引文件必须是CSV或JSONL格式。

#### JSONL

对于图像分割，JSONL 索引文件的要求如下：

- 每个数据项都是一个独立的 JSON 对象，位于单独的一行。
- 键/值对 `image_gcs_uri` 是图像的 Cloud 存储路径。
- 键/值对 `category_mask_uri` 是 PNG 格式掩模图像的 Cloud 存储路径。
- 键/值对 `'annotation_spec_colors'` 是一个列表，将掩模颜色映射到一个标签。
  - 键/值对 `display_name` 是像素颜色掩码的标签。
  - 键/值对 `color` 是对应标签掩码的 RGB 归一化像素值（介于 0 和 1 之间）。

    { 'image_gcs_uri': image, 'segmentation_annotations': { 'category_mask_uri': mask_image, 'annotation_spec_colors' : [ { 'display_name': label, 'color': {"red": value, "blue", value, "green": value} }, ...] }
    
*注*: 字典键字段也可以采用驼峰命名法。例如，'image_gcs_uri' 也可以是 'imageGcsUri'。

云存储训练数据的位置。

现在将变量“IMPORT_FILE”的值设置为云存储中JSONL索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://ucaip-test-us-central1/dataset/isg_data.jsonl"

快速查看您的数据

您将使用存储在公共云存储桶中的未知数据集的一个版本，使用一个JSONL索引文件。

首先快速查看数据。您可以通过计算JSONL索引文件中对象的数量（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的 Vertex 数据集资源中。使用这个辅助函数 `import_data` 来导入数据。该函数执行以下操作：

- 使用 `Dataset` 客户端。
- 调用客户端方法 `import_data`，并传入以下参数：
 - `name`：您给 `Dataset` 资源的可读名称（例如，unknown）。
 - `import_configs`：导入配置。

- `import_configs`：一个包含字典的 Python 列表，其中包含键/值条目：
 - `gcs_sources`：指向一个或多个索引文件路径的 URI 列表。
 - `import_schema_uri`：标识标签类型的模式。

`import_data()` 方法返回一个长时间运行的 `operation` 对象。这个过程需要几分钟来完成。如果您在进行实时教程，这将是一个很好的时间来提问或休息一下。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

## 训练模型

现在使用您的 Vertex `Dataset` 资源训练一个 AutoML 图像分割模型。要训练模型，请按照以下步骤操作：

1. 为 `Dataset` 资源创建一个 Vertex 训练流水线。
2. 执行流水线以开始训练。

### 创建训练流水线

您可能会问，我们用流水线做什么？当工作（如训练）具有多个步骤时，通常按顺序进行：执行步骤A，执行步骤B等等时，您通常会使用流水线。通过将这些步骤放入流水线中，我们可以获得以下好处：

1. 可重用于后续训练工作。
2. 可以将其容器化并作为批处理作业运行。
3. 可以分布式处理。
4. 所有步骤都与同一流水线作业关联，以追踪进度。

请使用此辅助函数`create_pipeline`，它接受以下参数：

- `pipeline_name`：流水线作业的易读名称。
- `model_name`：模型的易读名称。
- `dataset`：Vertex完全限定数据集标识符。
- `schema`：数据集标记（注释）训练模式。
- `task`：描述训练作业要求的字典。

辅助函数调用`Pipeline`客户端服务的`create_pipeline`方法，该方法接受以下参数：

- `parent`：用于您的`Dataset`、`Model`和`Endpoint`资源的Vertex位置根路径。
- `training_pipeline`：流水线训练作业的完整规范。

现在让我们深入了解构建`training_pipeline`规范的*最低*要求：

- `display_name`：流水线作业的易读名称。
- `training_task_definition`：数据集标记（注释）训练模式。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的易读名称。
- `input_data_config`：数据集规范。
  - `dataset_id`：仅Vertex数据集标识符（非完全限定）--这是完全限定标识符的最后一部分。
  - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比由AutoML自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务要求

接下来，构建任务要求。与其他参数不同，需要一个 Google protobuf Struct，而不是 Python（类似 JSON）字典，来填充`task`字段。可以使用`json_format.ParseDict`方法进行转换。

您需要指定的最小字段包括：

- `budget_milli_node_hours`：用于模型训练的最大计费时间，其中1000 = 1小时。
- `model_type`：部署模型的类型：
  - `CLOUD_HIGH_ACCURACY_1`：用于部署到 Google Cloud 并优化准确性。
  - `CLOUD_LOW_LATENCY_1`：用于部署到 Google Cloud 并优化延迟（响应时间）。

最后，通过调用帮助函数`create_pipeline`来创建管道，该函数将返回一个训练管道对象的实例。

In [ ]:
PIPE_NAME = "unknown_pipe-" + TIMESTAMP
MODEL_NAME = "unknown_model-" + TIMESTAMP

task = json_format.ParseDict(
    {"budget_milli_node_hours": 2000, "model_type": "CLOUD_LOW_ACCURACY_1"}, Value()
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取培训管道的信息

现在只获取此培训管道实例的管道信息。通过调用作业客户服务的`get_training_pipeline`方法，使用以下参数来获取此作业的作业信息：

- `name`：Vertex的完全限定管道标识符。

当模型训练完成时，管道状态将为`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

# 部署

训练上述模型可能需要长达30分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time` 减去 `start_time` 来计算训练模型所用的实际时间。对于您的模型，您需要知道完全限定的 Vertex 模型资源标识符，即管道服务分配给它的标识符。您可以从返回的管道实例中获得此信息，即字段`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

## 模型信息

现在您的模型已经训练完毕，您可以获取关于您的模型的一些信息。

评估模型资源

现在找出模型服务认为您的模型有多好。作为训练的一部分，数据集的一部分被保留作为测试（预留）数据，由管道服务用于评估模型。

### 列出所有切片的评估

使用这个辅助函数 `list_model_evaluations`，它接受以下参数：

- `name`：`Model` 资源的 Vertex 完全合格的模型标识符。

这个辅助函数使用模型客户端服务的 `list_model_evaluations` 方法，该方法使用同样的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估 -- 你可能只有一个，然后我们打印出每个评估中每个指标的所有键名称，对于一个小集合（`confidenceMetricsEntries`），您将打印出结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("confidenceMetricsEntries", metrics["confidenceMetricsEntries"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 批量预测模型部署

现在部署您为批量预测创建的经过训练的Vertex `Model`资源。这与部署用于按需预测的`Model`资源有所不同。

对于在线预测，您需要：

1. 创建一个用于部署`Model`资源的`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

3. 向`Endpoint`资源发出在线预测请求。

对于批量预测，您需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求预配资源。

3. 将批量预测请求的结果返回给调用方。

4. 作业服务将取消为批量预测请求预配的资源。

进行批量预测请求，现在对部署的模型进行批量预测。

获取测试项目

现在对您的 Vertex 模型进行批量预测。您将使用数据集中的任意示例作为测试项目。不必担心这些示例可能已用于训练模型 - 我们只是想展示如何进行预测。

In [ ]:
import json

test_items = !gsutil cat $IMPORT_FILE | head -n2
test_data_1 = test_items[0].replace("'", '"')
test_data_1 = json.loads(test_data_1)
test_data_2 = test_items[0].replace("'", '"')
test_data_2 = json.loads(test_data_2)
try:
    test_item_1 = test_data_1["image_gcs_uri"]
    test_label_1 = test_data_1["segmentation_annotation"]["annotation_spec_colors"]
    test_item_2 = test_data_2["image_gcs_uri"]
    test_label_2 = test_data_2["segmentation_annotation"]["annotation_spec_colors"]
except:
    test_item_1 = test_data_1["imageGcsUri"]
    test_label_1 = test_data_1["segmentationAnnotation"]["annotationSpecColors"]
    test_item_2 = test_data_2["imageGcsUri"]
    test_label_2 = test_data_2["segmentationAnnotation"]["annotationSpecColors"]

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

复制测试项目

对于批量预测，您需要将测试项目复制到您的云存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_NAME/$file_1
! gsutil cp $test_item_2 $BUCKET_NAME/$file_2

test_item_1 = BUCKET_NAME + "/" + file_1
test_item_2 = BUCKET_NAME + "/" + file_2

### 制作批量输入文件

现在制作一个批量输入文件，您将存储在本地云存储桶中。批量输入文件可以是 CSV 或 JSONL 格式。在本教程中，您将使用 JSONL 文件。对于 JSONL 文件，您将为每个数据项（实例）的每一行制作一个字典条目。该字典包含以下键/值对：

- `content`：图像的云存储路径。
- `mime_type`：内容类型。在我们的示例中，它是一个 `jpeg` 文件。

例如：

                        {'content': '[your-bucket]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 计算实例扩展

您在处理批量预测请求时有几种计算实例扩展的选择：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：批量预测请求被分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同数量的节点。当模型首次部署到实例上时，会预配固定数量的计算实例，并且批量预测请求会均匀分布在它们之间。

- 自动扩展：批量预测请求被分配到可扩展数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）数量设置为模型首次部署时预配和取消预配的实例数，并将计算实例的最大（`MAX_NODES`）数量设置为根据负载情况进行预配。

计算实例的最小数量对应于字段`min_replica_count`，计算实例的最大数量对应于字段`max_replica_count`，在您后续的部署请求中提到。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 发送批量预测请求

现在您的两个测试项的批次已准备就绪，让我们进行批量请求。使用以下参数的辅助函数 `create_batch_prediction_job`:

- `display_name`: 预测作业的人类可读名称。
- `model_name`: `Model` 资源的 Vertex 完全限定标识符。
- `gcs_source_uri`: 输入文件的 Cloud Storage 路径 -- 您在上面创建的。
- `gcs_destination_output_uri_prefix`: 服务将把预测写入的 Cloud Storage 路径。
- `parameters`: 用于提取预测结果的其他过滤参数。

辅助函数调用作业客户端服务的 `create_batch_prediction_job` 方法，带有以下参数:

- `parent`: 数据集、模型和管道资源的 Vertex 根路径。
- `batch_prediction_job`: 批处理预测作业的规范。

现在让我们深入讨论 `batch_prediction_job` 规范:

- `display_name`: 预测批次作业的人类可读名称。
- `model`: `Model` 资源的 Vertex 完全限定标识符。
- `dedicated_resources`: 为批量预测作业分配的计算资源。
  - `machine_spec`: 要配置的计算实例。使用之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `starting_replica_count`: 初始化要配置的计算实例数量，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`: 要扩展到的最大计算实例数量，您之前设置为变量 `MAX_NODES`。
- `model_parameters`: 用于提取预测结果的其他过滤参数。 *注意*，图像分割模型不支持额外参数。
- `input_config`: 需要预测的实例的输入来源和格式类型。
 - `instances_format`: 批处理预测请求文件的格式：只支持 `jsonl`。
 - `gcs_source`: 一个或多个 Cloud Storage 路径列表，用于批量预测请求。
- `output_config`: 预测的输出目的地和格式。
 - `prediction_format`: 批处理预测响应文件的格式：只支持 `jsonl`。
 - `gcs_destination`: 预测的输出目的地。

此调用是异步操作。您将从响应对象中打印一些选择字段，包括:

- `name`: 分配给批量预测作业的 Vertex 完全限定标识符。
- `display_name`: 预测批次作业的人类可读名称。
- `model`: 模型资源的 Vertex 完全限定标识符。
- `generate_explanations`: 是否提供了随预测一起的 True/False 解释（可解释性）。
- `state`: 预测作业的状态（等待中、运行中等）。

由于此调用将花费一些时间来执行，您可能会收到 `JobState.JOB_STATE_PENDING` 作为 `state`。

In [ ]:
BATCH_MODEL = "unknown_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "jsonl"
OUT_FORMAT = "jsonl"  # [jsonl]

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME, None
)

现在获取您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取批量预测作业的信息

请使用以下辅助函数 `get_batch_prediction_job`，并传入以下参数：

- `job_name`：用于表示批量预测作业的 Vertex 完全限定标识符。

该辅助函数将调用作业客户端服务的 `get_batch_prediction_job` 方法，并传入以下参数：

- `name`：用于表示批量预测作业的 Vertex 完全限定标识符。在本教程中，您将传入您批量预测作业的 Vertex 完全限定标识符 -- `batch_job_id`。

该辅助函数将返回存储预测结果的 Cloud 存储路径，即 `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取预测结果

当批量预测处理完成时，作业状态将是`JOB_STATE_SUCCEEDED`。

最后，您可以查看存储在您设置为输出的Cloud Storage路径上的预测结果。预测结果将以JSONL格式存储，这是您在创建批量预测作业时指定的，在一个以`prediction`开头的子文件夹下，该文件夹中将有一个名为`predictions*.jsonl`的文件。

现在显示（cat）内容。您将看到多个JSON对象，每个对象代表一个预测结果。

第一个字段`ID`是您进行预测的图像文件，第二个字段`annotations`是预测结果，进一步分解为：

- `confidenceMask`：PNG像素掩模，指示每个像素的预测置信度。
- `categoryMask`：PNG像素掩模，指示每个像素的预测结果。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)

清理

要清理此项目中使用的所有 GCP 资源，您可以[删除用于本教程的 GCP 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 管道
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME